In [1]:
import numpy as np
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [2]:
raw_data = np.loadtxt("diabetes.csv.gz", delimiter=',',dtype=np.float32)
X = raw_data[:,:-1]
y = raw_data[:,[-1]]
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)
print(x_train.shape)
print(y_train.shape)

(531, 8)
(531, 1)


In [3]:
class DiabetesDataset(Dataset):
    def __init__(self, data,label):
        self.len = data.shape[0]
        self.x_data = torch.from_numpy(data)
        self.y_data = torch.from_numpy(label)
        
    def __len__(self):
        return self.len
    
    def __getitem__(self,index):
        return self.x_data[index], self.y_data[index]
    

In [4]:
train_dataset = DiabetesDataset(x_train,y_train)
train_loader = DataLoader(dataset=train_dataset,
                         batch_size=32,
                         shuffle=True,
                         num_workers=0)


In [5]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(8, 6)
        self.linear2 = torch.nn.Linear(6, 4)
        self.linear3 = torch.nn.Linear(4, 2)
        self.linear4 = torch.nn.Linear(2, 1)
        self.sigmoid = torch.nn.Sigmoid()
 
    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        x = self.sigmoid(self.linear4(x))
        return x
 
 
model = Model()


In [6]:
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SDG(model.parameters(), lr=0.01)

In [7]:
def train(epoch):
    train_loss = 0.0
    count = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        y_pred = model(inputs)
 
        loss = criterion(y_pred, labels)
 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        count = i
 
    if epoch%2000 == 1999:
        print("train loss:", train_loss/count)


In [8]:
def test():
    with torch.no_grad():
        y_pred = model(x_test)
        y_pred_label = torch.where(y_pred>=0.5,torch.tensor([1.0]),torch.tensor([0.0]))
        acc = torch.eq(y_pred_label, y_test).sum().item() / y_test.size(0)
        print("test acc:", acc)


In [9]:
if __name__ == '__main__':
    for epoch in range(10000):
        train(epoch)
        if epoch%2000==1999:
            test()

train loss: 0.3571573793888092
test acc: 0.75
train loss: 0.3345256522297859
test acc: 0.7456140350877193
train loss: 0.3347781663760543
test acc: 0.7412280701754386
train loss: 0.3438312625512481
test acc: 0.7587719298245614
train loss: 0.3068920513615012
test acc: 0.7587719298245614
